In [1]:
%matplotlib inline
import pylab
import numpy as np
import nengo
import nengo_spa as spa


In [13]:
n_items = 25

fan1 = []
for i in range(int(n_items/5)):
    fan1.append(('F1_%d' % (2*i), 'F1_%d'% (2*i+1)))
fan2 = []
for i in range(int(n_items/5)):
    fan2.append(('F2_%d' % (4*i), 'F2_%d'% (4*i+1)))
    fan2.append(('F2_%d' % (4*i), 'F2_%d'% (4*i+2)))
    fan2.append(('F2_%d' % (4*i+3), 'F2_%d'% (4*i+2)))
    fan2.append(('F2_%d' % (4*i+3), 'F2_%d'% (4*i+1)))
pairs = fan1+fan2    
    
D = 512
vocab = spa.Vocabulary(D)
items = set()
for i1, i2 in fan1+fan2:
    items.add(i1)
    items.add(i2)
for item in items:
    vocab.populate(item)


In [49]:
seed = 1
rng = np.random.RandomState(seed=seed)
noise_sd = 1.0
eval_points = []
target = []
n_samples = 50000
for i in range(n_samples):
    pair = pairs[rng.randint(len(pairs))]
    pt = vocab.parse('%s + %s' % (pair[0], pair[1])).v
    target.append(pt)
    eval_points.append(pt+rng.normal(size=pt.shape)*noise_sd)


In [50]:
n_cue = 1000
n_result = 100
model = nengo.Network()
with model:
    cue = nengo.Ensemble(n_neurons=n_cue, dimensions=D)
    result = nengo.Ensemble(n_neurons=n_result, dimensions=D)
    
    c = nengo.Connection(cue, result, eval_points=eval_points, function=target)
    

In [51]:
sim = nengo.Simulator(model)
dec = sim.data[c].weights.T

Building finished in 0:00:06.                                                   


In [52]:
fan1_values = []
for pair in fan1:
    pt = vocab.parse('%s + %s' % (pair[0], pair[1])).v
    fan1_values.append(pt)
fan1_values = np.array(fan1_values)
_, activity = nengo.utils.ensemble.tuning_curves(cue, sim, inputs=fan1_values)
result = np.dot(activity, dec)
score = []
for i in range(5):
    s = np.dot(fan1_values[i], result[i])
    score.append(s)
print(score)
print(np.mean(score))

[0.30547096461100909, 0.19562748409170078, 0.16342564428698803, 0.2383384567702744, 0.19239606412664889]
0.219051722777


In [53]:
fan2_values = []
for pair in fan2:
    pt = vocab.parse('%s + %s' % (pair[0], pair[1])).v
    fan2_values.append(pt)
fan2_values = np.array(fan2_values)
_, activity = nengo.utils.ensemble.tuning_curves(cue, sim, inputs=fan2_values)
result = np.dot(activity, dec)
score = []
for i in range(5*4):
    s = np.dot(fan2_values[i], result[i])
    score.append(s)
print(score)
print(np.mean(score))

[0.38270831029767927, 0.35170639756899064, 0.319083301977727, 0.34477819287286271, 0.43021720107332162, 0.38633115515495331, 0.42102717852486737, 0.4247447136611352, 0.21737620466452381, 0.28317256618336917, 0.31092296409955367, 0.25621396643438477, 0.40125536794937999, 0.354453653589128, 0.28937346436592132, 0.28629194398535829, 0.26706276866821255, 0.19258982114537121, 0.36568464110828264, 0.39109764259740443]
0.333804572796
